In [1]:
import os
os.chdir('../')

In [2]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file

DATA_CONFIG_FILE_PATH = os.environ['DATA_CONFIG_FILE_PATH']

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataPreprocessingConfig:
    root_dir: Path
    raw_data_path: Path
    question_key: str
    answer_key: str

In [4]:
from src.utils.common import read_yaml

class ConfigurationManager:
    def __init__(self,
                 config_filepath = DATA_CONFIG_FILE_PATH):

        self.config = read_yaml(Path(config_filepath))


    def get_data_preprocessing_config(self) -> DataPreprocessingConfig:
        config = self.config.data_preprocessing


        data_preprocessing_config = DataPreprocessingConfig(
            root_dir=config.root_dir,
            raw_data_path=config.raw_data_path,
            question_key=config.question_key,
            answer_key=config.answer_key
        )

        return data_preprocessing_config

In [5]:
import pandas as pd
from src.logging import logger
from datasets import Dataset

class DataPreprocessing:
    def __init__(self, config: DataPreprocessingConfig):
        self.config = config
        
    def __form_finetuning_dataset(self, dataset_dict: dict, question_key: str, answer_key: str) -> Dataset:
        instruction_template = """{question}"""

        prompt_template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

        ### Instruction:

        {instruction}

        ### Response:\n"""

        num_samples = len(dataset_dict[question_key])
        finetuning_dataset_list = []
        for i in range(num_samples):
            question = dataset_dict[question_key][i]
            instruction = instruction_template.format(question=question)
            prompt = prompt_template.format(instruction=instruction)
            response = dataset_dict[answer_key][i] + "\n### End"
            text = prompt + response
            finetuning_dataset_list.append({"instruction": instruction, "response": response, "text": text})

        finetuning_dataset = Dataset.from_list(finetuning_dataset_list)

        return finetuning_dataset
    

    def convert(self):
        instruction_dataset_df = pd.read_csv(self.config.raw_data_path)
        instruction_dataset_dict = instruction_dataset_df.to_dict()
        logger.info("Successfully loaded the raw dataset")

        finetuning_dataset = self.__form_finetuning_dataset(instruction_dataset_dict, question_key = self.config.question_key, answer_key = self.config.answer_key)
        finetuning_dataset_df = finetuning_dataset.to_pandas()
        finetuning_dataset_df.to_csv(os.path.join(self.config.root_dir,"finetuning_dataset.csv"), index=False)
        logger.info("Successfully saved the finetuning dataset")

In [6]:
try:
    config = ConfigurationManager()
    data_preprocessing_config = config.get_data_preprocessing_config()
    data_preprocessing = DataPreprocessing(config=data_preprocessing_config)
    data_preprocessing.convert()
except Exception as e:
    raise e

[2023-11-30 18:58:49,070: INFO: common: yaml file: config\data-config.yaml loaded successfully]
[2023-11-30 18:58:49,233: INFO: 4118875211: Successfully loaded the raw data]
[2023-11-30 18:58:49,666: INFO: 4118875211: Successfully saved the finetuning dataset]
